In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers, optimizers, callbacks
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import warnings
import time
import abc
import os

# --- Environment Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

try:
    import cupy as cp
    CUPY_AVAILABLE = True
    print("CuPy found. GPU data acceleration is enabled.")
except ImportError:
    CUPY_AVAILABLE = False
    print("CuPy not found. Predictions will use CPU data, which may be slower.")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"TensorFlow is using GPU: {gpus[0].name}")
        TF_DEVICE = "/GPU:0"
    except RuntimeError as e:
        print(e); TF_DEVICE = "/CPU:0"
else:
    print("TensorFlow is using CPU."); TF_DEVICE = "/CPU:0"

# =========================================================================
# Helper and Base Classes
# =========================================================================
class MultiTargetModelWrapper:
    def __init__(self, models): self.models = models
    def predict(self, X):
        # This wrapper's predict now expects a NumPy array because of the fix below
        preds = [model.predict(X) for model in self.models]
        return np.column_stack(preds)

class BasePredictor(abc.ABC):
    def __init__(self, use_gpu=True):
        self.model = None; self.scalers = {}; self.best_features = None
        self.target_names = None; self.is_fitted = False
        self.device_type = 'cuda' if use_gpu and tf.config.list_physical_devices('GPU') else 'cpu'
    def create_advanced_features(self, df):
        df_features = df.copy()
        blend_cols = [col for col in df.columns if 'fraction' in col]
        if not blend_cols: return df_features
        df_features['blend_entropy'] = -np.sum(df[blend_cols] * np.log(df[blend_cols] + 1e-8), axis=1)
        return df_features
    def select_best_features(self, X, y, max_features=150):
        X = X.fillna(0)
        scores = pd.Series(mutual_info_regression(X, y.mean(axis=1), random_state=42), index=X.columns)
        return scores.nlargest(max_features).index.tolist()
    def fit(self, X, y):
        print(f"--- Fitting {self.__class__.__name__} ---")
        start_time = time.time()
        self.target_names = y.columns.tolist()
        X_processed = self.create_advanced_features(X)
        self.best_features = self.select_best_features(X_processed, y)
        X_selected = X_processed[self.best_features]
        self.scalers['feature_scaler'] = RobustScaler()
        X_scaled = pd.DataFrame(self.scalers['feature_scaler'].fit_transform(X_selected), columns=self.best_features)
        self.scalers['target_scaler'] = RobustScaler()
        y_scaled = pd.DataFrame(self.scalers['target_scaler'].fit_transform(y), columns=y.columns)
        X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
        self.model = self._train_model(X_train, y_train, X_val, y_val)
        self.is_fitted = True
        val_preds_scaled = self._predict_model(X_val)
        val_preds_orig = self.scalers['target_scaler'].inverse_transform(val_preds_scaled)
        y_val_orig = self.scalers['target_scaler'].inverse_transform(y_val)
        mape = mean_absolute_percentage_error(y_val_orig, val_preds_orig)
        print(f"Validation MAPE for {self.__class__.__name__}: {mape:.6f}")
        print(f"Completed fitting in {time.time() - start_time:.2f}s\n")
        return self
    def predict(self, X):
        if not self.is_fitted: raise RuntimeError("Must fit before predicting.")
        X_processed = self.create_advanced_features(X)
        X_selected = X_processed[self.best_features]
        X_scaled = pd.DataFrame(self.scalers['feature_scaler'].transform(X_selected), columns=self.best_features)
        preds_scaled = self._predict_model(X_scaled)
        preds_orig = self.scalers['target_scaler'].inverse_transform(preds_scaled)
        return pd.DataFrame(preds_orig, columns=self.target_names)
    @abc.abstractmethod
    def _train_model(self, X_train, y_train, X_val, y_val): pass
    @abc.abstractmethod
    def _predict_model(self, X): pass

# =========================================================================
# Model Implementations
# =========================================================================

class TensorFlowPredictor(BasePredictor):
    class ResidualBlock(layers.Layer):
        def __init__(self, units, dropout_rate, l2_reg, **kwargs):
            super().__init__(**kwargs)
            self.main_layers = [
                layers.Dense(units, activation='gelu', kernel_regularizer=regularizers.l2(l2_reg)),
                layers.BatchNormalization(), layers.Dropout(dropout_rate)
            ]
            self.skip_layers = [layers.Dense(units), layers.BatchNormalization()]
        def call(self, inputs):
            Z = inputs; Z = self.main_layers[0](Z); Z = self.main_layers[1](Z); Z = self.main_layers[2](Z)
            skip_Z = inputs; skip_Z = self.skip_layers[0](skip_Z); skip_Z = self.skip_layers[1](skip_Z)
            return layers.add([Z, skip_Z])
    class FeatureAttention(layers.Layer):
        def build(self, input_shape):
            num_features = input_shape[-1]
            self.dense1 = layers.Dense(max(1, num_features // 4), activation='tanh', name="attention_context")
            self.dense2 = layers.Dense(num_features, activation='softmax', name="attention_weights")
            super().build(input_shape)
        def call(self, inputs):
            context = self.dense1(inputs)
            attention_weights = self.dense2(context)
            return inputs * attention_weights
    def _create_model(self, input_shape, output_shape, wide_input_shape):
        l2_reg = 1e-5; dropout_rate = 0.2
        residual_units = [512, 512, 256, 256, 128]
        deep_input = layers.Input(shape=(input_shape,), name='deep_input')
        Z = deep_input
        for units in residual_units: Z = self.ResidualBlock(units, dropout_rate, l2_reg)(Z)
        Z = self.FeatureAttention()(Z)
        wide_input = layers.Input(shape=(wide_input_shape,), name='wide_input')
        combined = layers.concatenate([Z, wide_input])
        output = layers.Dense(256, activation='gelu')(combined)
        output = layers.BatchNormalization()(output)
        output = layers.Dropout(dropout_rate)(output)
        output = layers.Dense(output_shape, name='output')(output)
        return Model(inputs=[deep_input, wide_input], outputs=output)
    def _train_model(self, X_train, y_train, X_val, y_val):
        with tf.device(TF_DEVICE):
            input_shape = X_train.shape[1]
            wide_feature_count = max(1, input_shape // 10)
            model = self._create_model(input_shape, y_train.shape[1], wide_input_shape=wide_feature_count)
            optimizer = optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-5)
            model.compile(loss='mean_squared_error', optimizer=optimizer)
            callbacks_list = [
                callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
                callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15)
            ]
            X_train_deep, X_train_wide = X_train.values, X_train.values[:, :wide_feature_count]
            X_val_deep, X_val_wide = X_val.values, X_val.values[:, :wide_feature_count]
            model.fit([X_train_deep, X_train_wide], y_train, validation_data=([X_val_deep, X_val_wide], y_val),
                      epochs=500, batch_size=64, callbacks=callbacks_list, verbose=1)
        return model
    def _predict_model(self, X):
        with tf.device(TF_DEVICE):
            wide_feature_count = max(1, X.shape[1] // 10)
            X_deep, X_wide = X.values, X.values[:, :wide_feature_count]
            return self.model.predict([X_deep, X_wide], verbose=0)

class XGBoostPredictor(BasePredictor):
    def _train_model(self, X_train, y_train, X_val, y_val):
        params = {'n_estimators': 2000, 'learning_rate': 0.05, 'max_depth': 7, 'subsample': 0.8,
                  'colsample_bytree': 0.8, 'random_state': 42, 'tree_method': 'hist',
                  'device': 'cuda' if self.device_type == 'cuda' else 'cpu', 'early_stopping_rounds': 50}
        models = [xgb.XGBRegressor(**params).fit(X_train, y_train.iloc[:, i], eval_set=[(X_val, y_val.iloc[:, i])], verbose=False) for i in range(y_train.shape[1])]
        return MultiTargetModelWrapper(models)
    def _predict_model(self, X):
        # ### THIS IS THE FIX ###
        # If X is a CuPy array, convert it to NumPy before prediction.
        if CUPY_AVAILABLE and isinstance(X, cp.ndarray):
            X = X.get()
        return self.model.predict(X)

class LightGBMPredictor(BasePredictor):
    def _train_model(self, X_train, y_train, X_val, y_val):
        params = {'n_estimators': 2500, 'learning_rate': 0.05, 'num_leaves': 40, 'max_depth': 8,
                  'max_bin': 128, 'random_state': 42,
                  'device': 'gpu' if self.device_type == 'cuda' else 'cpu', 'verbose': -1}
        models = [lgb.LGBMRegressor(**params).fit(X_train, y_train.iloc[:, i], eval_set=[(X_val, y_val.iloc[:, i])], callbacks=[lgb.early_stopping(50, verbose=False)]) for i in range(y_train.shape[1])]
        return MultiTargetModelWrapper(models)
    def _predict_model(self, X):
        # ### THIS IS THE FIX ###
        # If X is a CuPy array, convert it to NumPy before prediction.
        if CUPY_AVAILABLE and isinstance(X, cp.ndarray):
            X = X.get()
        return self.model.predict(X)

class CatBoostPredictor(BasePredictor):
    def _train_model(self, X_train, y_train, X_val, y_val):
        params = {'iterations': 2000, 'learning_rate': 0.05, 'depth': 7, 'random_seed': 42,
                  'task_type': 'GPU' if self.device_type == 'cuda' else 'CPU',
                  'verbose': 0, 'allow_writing_files': False, 'early_stopping_rounds': 50}
        models = [cb.CatBoostRegressor(**params).fit(X_train, y_train.iloc[:, i], eval_set=[(X_val, y_val.iloc[:, i])], verbose=False) for i in range(y_train.shape[1])]
        return MultiTargetModelWrapper(models)
    def _predict_model(self, X):
        # ### THIS IS THE FIX ###
        # If X is a CuPy array, convert it to NumPy before prediction.
        if CUPY_AVAILABLE and isinstance(X, cp.ndarray):
            X = X.get()
        return self.model.predict(X)

class RandomForestPredictor(BasePredictor):
    def _train_model(self, X_train, y_train, X_val, y_val):
        model = RandomForestRegressor(n_estimators=150, max_depth=12, min_samples_leaf=3, random_state=42, n_jobs=-1)
        return MultiOutputRegressor(model, n_jobs=-1).fit(X_train, y_train)
    def _predict_model(self, X): return self.model.predict(X)

# =========================================================================
# Ensemble Predictor
# =========================================================================
class EnsemblePredictor:
    def __init__(self, models, weights):
        if not models or sum(weights) == 0: raise ValueError("Models and weights required.")
        self.models = models
        self.weights = np.array(weights) / np.sum(weights)
        self.target_names = None
    def fit(self, X, y):
        print("====== STARTING ENSEMBLE TRAINING ======")
        self.target_names = y.columns.tolist()
        for model in self.models:
            model.fit(X.copy(), y.copy())
        print("====== ENSEMBLE TRAINING COMPLETE ======")
        return self
    def predict(self, X):
        print("\n====== GENERATING ENSEMBLE PREDICTIONS ======")
        all_preds = [m.predict(X.copy()) for m in self.models]
        all_preds_np = [p.values if isinstance(p, pd.DataFrame) else p for p in all_preds]
        ensembled = np.average(all_preds_np, axis=0, weights=self.weights)
        return pd.DataFrame(ensembled, columns=self.target_names)

# =========================================================================
# Main Execution Block
# =========================================================================
def main():
    print("🚀 Starting Fuel Blending ML Pipeline")
    try:
        train_df = pd.read_csv('/kaggle/input/training/train.csv')
        test_df = pd.read_csv('/kaggle/input/testing/test.csv')
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please check file paths.")
        return
    
    target_columns = [col for col in train_df.columns if 'BlendProperty' in col]
    feature_columns = [col for col in train_df.columns if col not in target_columns and 'ID' not in col]
    
    X_train, y_train = train_df[feature_columns], train_df[target_columns]
    X_test = test_df[feature_columns]

    models_to_train = [
        TensorFlowPredictor(),
        XGBoostPredictor(),
        LightGBMPredictor(),
        CatBoostPredictor(),
        RandomForestPredictor()
    ]
    ensemble_weights = [0.40, 0.20, 0.20, 0.15, 0.05]

    ensemble_model = EnsemblePredictor(models=models_to_train, weights=ensemble_weights)
    ensemble_model.fit(X_train, y_train)

    predictions = ensemble_model.predict(X_test)
    submission = pd.DataFrame({'ID': test_df.get('ID', test_df.index)})
    submission = pd.concat([submission, predictions], axis=1)
    submission.to_csv('submission.csv', index=False)
    
    print("\n💾 Submission file 'submission.csv' saved successfully.")
    print(submission.head())

if __name__ == "__main__":
    main()

CuPy found. GPU data acceleration is enabled.
TensorFlow is using GPU: /physical_device:GPU:0
🚀 Starting Fuel Blending ML Pipeline
====== STARTING ENSEMBLE TRAINING ======
--- Fitting TensorFlowPredictor ---
Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step - loss: 1.3492 - val_loss: 0.5099 - learning_rate: 0.0010
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4994 - val_loss: 0.5015 - learning_rate: 0.0010
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4254 - val_loss: 0.4966 - learning_rate: 0.0010
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3872 - val_loss: 0.4896 - learning_rate: 0.0010
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3551 - val_loss: 0.4818 - learning_rate: 0.0010
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3224 - val_loss: 0.4767 - learning_rate: 0.0010
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2935 - val_loss: 0.4627 - learning_rate: 0.0010
Epoch 8/500
25/25 ━━━━━━━━━